In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob 
import os
import plotly.graph_objects as go
import plotly.express as px
import plotly
plotly.io.orca.config.executable = '/home/remy/ns-3/orca'

In [19]:
filenames = list(glob.iglob("OUTPUT/Simple/*.csv"))
print(len(filenames))

2160


In [3]:
pd_dataframes = {'continuous': [], 'phi': [], 'summary': [], 'positions': [], 'initialpositions': [], 'finished': [], 'lost': []}
for filename in filenames:
    full_path = os.path.abspath(filename)
    path = os.path.dirname(full_path)
    basename = os.path.basename(filename)
    finished_filename = path + "/" + "finished_" + "_".join(basename.split("_")[1:])
    #if not os.path.isfile(finished_filename):
    #    print("%s does not exist ! " % os.path.basename(finished_filename))
    #    print("%s does exist" % basename)
    #    continue
    try:
        data_file_delimiter = ','
        header = []
        with open(full_path, 'r') as temp_f:
            header = temp_f.readline().rstrip('\n').split(data_file_delimiter) + ['empty']
        df = pd.read_csv(full_path, names=header, skiprows=1)
        basename = os.path.basename(filename)
        name = basename.split("_")[0]
        arguments = basename.rstrip(".csv").split("_")[3:]
        it = iter(arguments)
        for val in it:
            parameter = val.split("=")[0]
            value = val.split("=")[1]
            df[parameter] = value
        pd_dataframes[name].append(df)
    except Exception as e:
        print(e)
        #print(full_path)
        print("Error: "+ full_path)

In [4]:
dfo = {}
for name in pd_dataframes:
    if(pd_dataframes[name]):
        dfo[name] = pd.concat(pd_dataframes[name])
df = dfo.copy()

In [5]:
df['continuous']['Throughput'] = (df['continuous']['Rx Bytes']*8)

In [6]:
df['positions'] = df['positions'].set_index(['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'dataRate', 'udp', 'RngRun'])
new_df = df['continuous'].join(df['positions'], how='left', on=['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'dataRate', 'udp', 'RngRun'], lsuffix='_left', rsuffix='_right')
df['summary'] = df['summary'].set_index(['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'dataRate', 'udp', 'RngRun', 'Flow', 'Source Address', 'Destination Address'])
new_df = new_df.join(df['summary'], how='left', on=['N', 'distance', 'wifiManager', 'simulationTime', 'agentType', 'dataRate', 'udp', 'RngRun', 'Flow', 'Source Address', 'Destination Address'], rsuffix='_total')

In [7]:
convergence_time = []
g = None
for name, group in new_df.groupby(['N', 'distance', 'simulationTime', 'wifiManager', 'agentType', 'dataRate', 'udp', 'RngRun', 'Flow']):
    antenna_convergence = group['last_change'].mean()
    throughput_convergence = None
    full_rx = None
    g = group
    for agent_id, agent_group in group.groupby(['agent_id']):
        if agent_id == 0:
            low, high = 0.95*(agent_group['Rx Bytes'].values[-1]), 1.05*(agent_group['Rx Bytes'].values[-1])
            agent_group['between'] = agent_group['Rx Bytes'].between(low, high)
            agent_group['between'] = ~agent_group['between']
            k = agent_group['between'][agent_group['between']].index.max()
            #k = agent_group['between'][agent_group['between']==False].last_valid_index()
            #print(k)
            if (k is None) or (pd.isna(k)):
                throughput_convergence = 0
            else:
                throughput_convergence = agent_group.loc[k]['Time']
            if name[4] == "Drone":
                full_rx = agent_group[agent_group['Time'] >= antenna_convergence]['Throughput'].mean()
            else:
                full_rx = agent_group['Throughput'].mean()
    convergence_time.append(list(name) + [throughput_convergence, antenna_convergence, full_rx])

In [8]:
conv = pd.DataFrame(convergence_time, columns=['N', 'distance', 'simulationTime', 'wifiManager', 'agentType', 'dataRate', 'udp', 'RngRun', 'Flow', 'Throughput Converge', 'Antenna Converge', 'Throughput'])
full = conv.copy()
conv = conv[conv['agentType'] == 'Drone']
description = conv.groupby(['wifiManager']).describe()

In [14]:
# import plotly.express as px

distance = '100'
filtered = conv[(conv['distance'] == distance)]

trace0 = go.Box(
    y=filtered['Antenna Converge'],
    x=filtered['wifiManager'],
    name='Orientation Convergence Time',
    boxmean=True
)

trace1 = go.Box(
    y=filtered['Throughput Converge'],
    x=filtered['wifiManager'],
    name='Throughput Convergence Time',
    boxmean=True
)

data = [trace0, trace1]
layout = go.Layout(
    #title = "foo",
    template = "simple_white",
    xaxis_title="Wifi Manager",
    yaxis=go.layout.YAxis(
        title='Time (s)',
        range=[-5, max(filtered['Throughput Converge'].max()*1.05, filtered['Antenna Converge'].max()*1.05)],
        zeroline=False
    ),
    boxmode='group'
)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend_orientation="h")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.show()
fig.write_image("convergence_simple_3dB.png")

In [10]:
conv[conv['Throughput'] <= 1000*1000*10]

,N,distance,simulationTime,wifiManager,agentType,dataRate,udp,RngRun,Flow,Throughput Converge,Antenna Converge,Throughput
269,2,300,100,ns3::IdealWifiManager,Drone,180Mbps,true,17,2,28,27.240,163770.810811
341,2,300,100,ns3::MinstrelHtWifiManager,Drone,180Mbps,true,1,2,25,25.445,0.000000


In [15]:
# import plotly.express as px

radius = '100'
filtered = full[(full['distance'] == radius)]
print(set(filtered['N']))
filtered = filtered[(filtered['N'] == '2')]
print(set(filtered['dataRate']))
filtered = filtered[(filtered['dataRate'] == '180Mbps')]
drone = filtered[(filtered['agentType'] == 'Drone')]
client = filtered[(filtered['agentType'] == 'Client')]

trace0 = go.Box(
    y=(client['Throughput']),
    x=client['wifiManager'],
    name='Client',
    boxmean=True
)

trace1 = go.Box(
    y=(drone['Throughput']),
    x=drone['wifiManager'],
    name='Drone',
    boxmean=True
)


data = [trace0, trace1]
layout = go.Layout(
    template = "simple_white",
    xaxis_title="Wifi Manager",
    yaxis=go.layout.YAxis(
        title='Throughput (bps)',
    ),
    boxmode='group',)

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend_orientation="h")
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGrey')
fig.show()
fig.write_image("débit_simple_3dB.png")

{'2'}
{'180Mbps'}


In [17]:
i = 0

for name, group in df['continuous'].groupby(['N', 'distance', 'dataRate', 'RngRun']):
    fig = go.Figure()
    for wifiManager in sorted(set(group['wifiManager'])):
        for agentType in sorted(set(group['agentType'])):
            local = group[(group['wifiManager'] == wifiManager) & (group['agentType'] == agentType)]
            str_group = name + (wifiManager, agentType)
            fig.add_trace(go.Scatter(x=local["Time"], y=local["Rx Bytes"]*8, name=" ".join(str_group)))
    fig.update_layout(
        title="", template="simple_white"
    )
    fig.update_layout(legend_orientation="h")
    fig.update_xaxes(title="Time (s)", showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig.update_yaxes(title="Throughput (bps)", showgrid=True, gridwidth=1, gridcolor='LightGrey')
    fig.show()
    if (i == 0):
        print("Writing the figure")
        fig.write_image("evolution.png")
        break
    i += 1

Writing the figure


In [13]:
filtered = df['positions'][df['positions']['agentType'] == "Drone"]
fig = px.scatter(filtered, x="angle", color="wifiManager")
fig.update_xaxes(range=[0, 360])
mesh = [-6.8,-6.8,-6.8,-6.8,-6.7,-6.7,-6.7,-6.6,-6.6,-6.6,-6.5,-6.5,-6.4,-6.4,-6.3,-6.3,-6.3,-6.2,-6.2,-6.1,-6.1,-6.0,-6.0,-5.9,-5.9,-5.8,-5.7,-5.7,-5.6,-5.5,-5.4,-5.3,-5.2,-5.1,-5.0,-4.9,-4.7,-4.6,-4.5,-4.4,-4.3,-4.2,-4.1,-4.0,-3.9,-3.8,-3.7,-3.6,-3.5,-3.4,-3.3,-3.1,-3.0,-2.8,-2.7,-2.5,-2.3,-2.2,-2.0,-1.8,-1.6,-1.5,-1.3,-1.1,-1.0,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.2,-0.1,-0.1,-0.1,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.1,-0.1,-0.1,-0.2,-0.2,-0.3,-0.4,-0.5,-0.6,-0.7,-0.8,-0.9,-1.0,-1.2,-1.3,-1.5,-1.7,-1.8,-2.0,-2.2,-2.3,-2.5,-2.7,-2.9,-3.0,-3.2,-3.3,-3.5,-3.6,-3.8,-3.9,-4.1,-4.2,-4.3,-4.4,-4.5,-4.6,-4.7,-4.7,-4.8,-4.9,-4.9,-5.0,-5.1,-5.2,-5.2,-5.3,-5.4,-5.5,-5.6,-5.6,-5.7,-5.8,-5.9,-6.0,-6.1,-6.2,-6.3,-6.4,-6.4,-6.5,-6.6,-6.7,-6.7,-6.8,-6.8,-6.9,-6.9,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-6.9,-6.9,-6.9,-6.8,-6.8,-6.7,-6.7,-6.6,-6.5,-6.5,-6.4,-6.3,-6.2,-6.2,-6.1,-6.0,-6.0,-5.9,-5.8,-5.8,-5.7,-5.6,-5.6,-5.5,-5.5,-5.4,-5.3,-5.3,-5.2,-5.1,-5.1,-5.0,-4.9,-4.9,-4.8,-4.7,-4.6,-4.6,-4.5,-4.4,-4.3,-4.2,-4.1,-4.0,-3.8,-3.7,-3.6,-3.4,-3.3,-3.1,-3.0,-2.8,-2.6,-2.4,-2.3,-2.1,-1.9,-1.7,-1.6,-1.4,-1.2,-1.1,-0.9,-0.8,-0.7,-0.5,-0.4,-0.3,-0.2,-0.2,-0.1,-0.1,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.2,-0.2,-0.3,-0.3,-0.4,-0.5,-0.6,-0.7,-0.8,-0.9,-1.1,-1.2,-1.4,-1.6,-1.7,-1.9,-2.1,-2.3,-2.4,-2.6,-2.8,-2.9,-3.1,-3.2,-3.4,-3.5,-3.6,-3.8,-3.9,-4.0,-4.2,-4.3,-4.4,-4.5,-4.7,-4.8,-4.9,-5.0,-5.1,-5.2,-5.3,-5.3,-5.4,-5.5,-5.5,-5.6,-5.7,-5.7,-5.8,-5.9,-5.9,-6.0,-6.1,-6.1,-6.2,-6.2,-6.3,-6.4,-6.4,-6.5,-6.6,-6.6,-6.7,-6.7,-6.7,-6.8,-6.8,-6.8,-6.8,-6.8,-6.8,-6.8,-6.8,-5.7,-5.7,-5.7,-5.7,-5.8,-5.8,-5.8,-5.9,-5.9,-6.0,-6.0,-6.1,-6.2,-6.3,-6.4,-6.5,-6.6,-6.7,-6.8,-7.0,-7.1,-7.2,-7.4,-7.5,-7.6,-7.7,-7.8,-7.9,-7.9,-7.9,-7.8,-7.7,-7.6,-7.4,-7.2,-7.0,-6.7,-6.5,-6.3,-6.1,-5.8,-5.7,-5.5,-5.3,-5.2,-5.1,-5.0,-4.8,-4.7,-4.5,-4.3,-4.1,-3.8,-3.5,-3.2,-2.9,-2.6,-2.3,-2.0,-1.7,-1.5,-1.3,-1.2,-1.1,-1.0,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.1,-0.1,-0.0,-0.0,-0.1,-0.2,-0.3,-0.5,-0.7,-1.0,-1.1,-1.3,-1.4,-1.5,-1.5,-1.6,-1.7,-1.8,-2.0,-2.3,-2.6,-2.9,-3.1,-3.3,-3.3,-3.2,-3.0,-2.8,-2.6,-2.5,-2.4,-2.4,-2.5,-2.6,-2.7,-2.9,-3.0,-3.1,-3.2,-3.2,-3.3,-3.3,-3.4,-3.5,-3.7,-3.9,-4.2,-4.6,-5.1,-5.6,-6.2,-6.8,-7.4,-7.9,-8.4,-8.9,-9.3,-9.7,-10.0,-10.3,-10.7,-11.2,-11.8,-12.5,-13.3,-14.2,-15.2,-16.2,-16.9,-17.2,-16.9,-16.2,-15.3,-14.4,-13.5,-12.7,-12.0,-11.5,-11.0,-10.6,-10.2,-10.0,-9.7,-9.6,-9.5,-9.4,-9.3,-9.3,-9.3,-9.4,-9.5,-9.6,-9.7,-9.8,-10.0,-10.1,-10.3,-10.5,-10.7,-10.9,-11.1,-11.3,-11.4,-11.6,-11.7,-11.8,-11.8,-11.9,-11.9,-11.9,-11.8,-11.7,-11.6,-11.5,-11.4,-11.2,-11.0,-10.8,-10.7,-10.5,-10.3,-10.1,-10.0,-9.8,-9.7,-9.6,-9.5,-9.4,-9.4,-9.4,-9.4,-9.4,-9.5,-9.6,-9.8,-10.0,-10.2,-10.6,-11.0,-11.4,-12.0,-12.7,-13.5,-14.3,-15.3,-16.2,-16.9,-17.2,-16.9,-16.2,-15.3,-14.3,-13.3,-12.5,-11.7,-11.1,-10.6,-10.2,-9.9,-9.5,-9.2,-8.8,-8.3,-7.8,-7.3,-6.7,-6.1,-5.6,-5.1,-4.6,-4.2,-3.9,-3.7,-3.5,-3.4,-3.3,-3.3,-3.3,-3.2,-3.2,-3.0,-2.9,-2.8,-2.6,-2.5,-2.4,-2.4,-2.5,-2.6,-2.8,-3.0,-3.1,-3.2,-3.2,-3.0,-2.8,-2.6,-2.3,-2.0,-1.8,-1.7,-1.6,-1.5,-1.4,-1.4,-1.3,-1.1,-1.0,-0.8,-0.5,-0.3,-0.2,-0.1,-0.0,0.0,-0.0,-0.1,-0.2,-0.4,-0.5,-0.6,-0.7,-0.8,-0.9,-1.0,-1.1,-1.2,-1.3,-1.5,-1.8,-2.0,-2.3,-2.6,-3.0,-3.3,-3.6,-3.9,-4.2,-4.4,-4.6,-4.7,-4.9,-5.0,-5.1,-5.2,-5.4,-5.5,-5.7,-5.9,-6.1,-6.3,-6.5,-6.7,-7.0,-7.2,-7.4,-7.6,-7.7,-7.8,-7.9,-7.9,-7.9,-7.9,-7.8,-7.7,-7.6,-7.4,-7.3,-7.2,-7.0,-6.9,-6.8,-6.6,-6.5,-6.4,-6.3,-6.2,-6.1,-6.0,-6.0,-5.9,-5.9,-5.8,-5.8,-5.7,-5.7,-5.7,-5.7]
mesh = [(8+x)*10 for x in mesh]
fig.add_trace(go.Scatter(
    x = list(range(360)),
    y = mesh
))

KeyError: 'agentType'

In [ ]:
fig = px.histogram(filtered, x="angle", nbins=80)
fig.update_xaxes(range=[0, 360])
fig.show()

In [ ]:
worse = 0
better = 0
mesh = [-6.8,-6.8,-6.8,-6.8,-6.7,-6.7,-6.7,-6.6,-6.6,-6.6,-6.5,-6.5,-6.4,-6.4,-6.3,-6.3,-6.3,-6.2,-6.2,-6.1,-6.1,-6.0,-6.0,-5.9,-5.9,-5.8,-5.7,-5.7,-5.6,-5.5,-5.4,-5.3,-5.2,-5.1,-5.0,-4.9,-4.7,-4.6,-4.5,-4.4,-4.3,-4.2,-4.1,-4.0,-3.9,-3.8,-3.7,-3.6,-3.5,-3.4,-3.3,-3.1,-3.0,-2.8,-2.7,-2.5,-2.3,-2.2,-2.0,-1.8,-1.6,-1.5,-1.3,-1.1,-1.0,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.2,-0.2,-0.1,-0.1,-0.1,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.1,-0.1,-0.1,-0.2,-0.2,-0.3,-0.4,-0.5,-0.6,-0.7,-0.8,-0.9,-1.0,-1.2,-1.3,-1.5,-1.7,-1.8,-2.0,-2.2,-2.3,-2.5,-2.7,-2.9,-3.0,-3.2,-3.3,-3.5,-3.6,-3.8,-3.9,-4.1,-4.2,-4.3,-4.4,-4.5,-4.6,-4.7,-4.7,-4.8,-4.9,-4.9,-5.0,-5.1,-5.2,-5.2,-5.3,-5.4,-5.5,-5.6,-5.6,-5.7,-5.8,-5.9,-6.0,-6.1,-6.2,-6.3,-6.4,-6.4,-6.5,-6.6,-6.7,-6.7,-6.8,-6.8,-6.9,-6.9,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-6.9,-6.9,-6.9,-6.8,-6.8,-6.7,-6.7,-6.6,-6.5,-6.5,-6.4,-6.3,-6.2,-6.2,-6.1,-6.0,-6.0,-5.9,-5.8,-5.8,-5.7,-5.6,-5.6,-5.5,-5.5,-5.4,-5.3,-5.3,-5.2,-5.1,-5.1,-5.0,-4.9,-4.9,-4.8,-4.7,-4.6,-4.6,-4.5,-4.4,-4.3,-4.2,-4.1,-4.0,-3.8,-3.7,-3.6,-3.4,-3.3,-3.1,-3.0,-2.8,-2.6,-2.4,-2.3,-2.1,-1.9,-1.7,-1.6,-1.4,-1.2,-1.1,-0.9,-0.8,-0.7,-0.5,-0.4,-0.3,-0.2,-0.2,-0.1,-0.1,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.1,-0.2,-0.2,-0.3,-0.3,-0.4,-0.5,-0.6,-0.7,-0.8,-0.9,-1.1,-1.2,-1.4,-1.6,-1.7,-1.9,-2.1,-2.3,-2.4,-2.6,-2.8,-2.9,-3.1,-3.2,-3.4,-3.5,-3.6,-3.8,-3.9,-4.0,-4.2,-4.3,-4.4,-4.5,-4.7,-4.8,-4.9,-5.0,-5.1,-5.2,-5.3,-5.3,-5.4,-5.5,-5.5,-5.6,-5.7,-5.7,-5.8,-5.9,-5.9,-6.0,-6.1,-6.1,-6.2,-6.2,-6.3,-6.4,-6.4,-6.5,-6.6,-6.6,-6.7,-6.7,-6.7,-6.8,-6.8,-6.8,-6.8,-6.8,-6.8,-6.8,-6.8,-5.7,-5.7,-5.7,-5.7,-5.8,-5.8,-5.8,-5.9,-5.9,-6.0,-6.0,-6.1,-6.2,-6.3,-6.4,-6.5,-6.6,-6.7,-6.8,-7.0,-7.1,-7.2,-7.4,-7.5,-7.6,-7.7,-7.8,-7.9,-7.9,-7.9,-7.8,-7.7,-7.6,-7.4,-7.2,-7.0,-6.7,-6.5,-6.3,-6.1,-5.8,-5.7,-5.5,-5.3,-5.2,-5.1,-5.0,-4.8,-4.7,-4.5,-4.3,-4.1,-3.8,-3.5,-3.2,-2.9,-2.6,-2.3,-2.0,-1.7,-1.5,-1.3,-1.2,-1.1,-1.0,-0.9,-0.8,-0.7,-0.6,-0.5,-0.4,-0.3,-0.1,-0.1,-0.0,-0.0,-0.1,-0.2,-0.3,-0.5,-0.7,-1.0,-1.1,-1.3,-1.4,-1.5,-1.5,-1.6,-1.7,-1.8,-2.0,-2.3,-2.6,-2.9,-3.1,-3.3,-3.3,-3.2,-3.0,-2.8,-2.6,-2.5,-2.4,-2.4,-2.5,-2.6,-2.7,-2.9,-3.0,-3.1,-3.2,-3.2,-3.3,-3.3,-3.4,-3.5,-3.7,-3.9,-4.2,-4.6,-5.1,-5.6,-6.2,-6.8,-7.4,-7.9,-8.4,-8.9,-9.3,-9.7,-10.0,-10.3,-10.7,-11.2,-11.8,-12.5,-13.3,-14.2,-15.2,-16.2,-16.9,-17.2,-16.9,-16.2,-15.3,-14.4,-13.5,-12.7,-12.0,-11.5,-11.0,-10.6,-10.2,-10.0,-9.7,-9.6,-9.5,-9.4,-9.3,-9.3,-9.3,-9.4,-9.5,-9.6,-9.7,-9.8,-10.0,-10.1,-10.3,-10.5,-10.7,-10.9,-11.1,-11.3,-11.4,-11.6,-11.7,-11.8,-11.8,-11.9,-11.9,-11.9,-11.8,-11.7,-11.6,-11.5,-11.4,-11.2,-11.0,-10.8,-10.7,-10.5,-10.3,-10.1,-10.0,-9.8,-9.7,-9.6,-9.5,-9.4,-9.4,-9.4,-9.4,-9.4,-9.5,-9.6,-9.8,-10.0,-10.2,-10.6,-11.0,-11.4,-12.0,-12.7,-13.5,-14.3,-15.3,-16.2,-16.9,-17.2,-16.9,-16.2,-15.3,-14.3,-13.3,-12.5,-11.7,-11.1,-10.6,-10.2,-9.9,-9.5,-9.2,-8.8,-8.3,-7.8,-7.3,-6.7,-6.1,-5.6,-5.1,-4.6,-4.2,-3.9,-3.7,-3.5,-3.4,-3.3,-3.3,-3.3,-3.2,-3.2,-3.0,-2.9,-2.8,-2.6,-2.5,-2.4,-2.4,-2.5,-2.6,-2.8,-3.0,-3.1,-3.2,-3.2,-3.0,-2.8,-2.6,-2.3,-2.0,-1.8,-1.7,-1.6,-1.5,-1.4,-1.4,-1.3,-1.1,-1.0,-0.8,-0.5,-0.3,-0.2,-0.1,-0.0,0.0,-0.0,-0.1,-0.2,-0.4,-0.5,-0.6,-0.7,-0.8,-0.9,-1.0,-1.1,-1.2,-1.3,-1.5,-1.8,-2.0,-2.3,-2.6,-3.0,-3.3,-3.6,-3.9,-4.2,-4.4,-4.6,-4.7,-4.9,-5.0,-5.1,-5.2,-5.4,-5.5,-5.7,-5.9,-6.1,-6.3,-6.5,-6.7,-7.0,-7.2,-7.4,-7.6,-7.7,-7.8,-7.9,-7.9,-7.9,-7.9,-7.8,-7.7,-7.6,-7.4,-7.3,-7.2,-7.0,-6.9,-6.8,-6.6,-6.5,-6.4,-6.3,-6.2,-6.1,-6.0,-6.0,-5.9,-5.9,-5.8,-5.8,-5.7,-5.7,-5.7,-5.7]
filtered = df['positions'][(df['positions']['agentType'] == "Drone") & (df['positions']['distance'] == "100")]
for name, group in filtered.groupby(['N', 'distance', 'wifiManager', 'RngRun']):
    angles = set(group['angle'])
    if len(angles) != 2:
        print("Erreur")
    angles = list(angles)
    gain = 0
    gain += mesh[int(angles[0])] + 3
    gain += mesh[int(angles[1])] + 3
    if (gain < 0.0):
        worse += 1
    elif (gain > 0.0):
        better += 1

In [ ]:
worse / (worse + better)

In [ ]:
worse

In [ ]:
better

In [ ]:
better / (worse + better)